In [1]:
import pdb
import math
import random

import gym
import numpy as np
import reward as tr
import reward.utils as U

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from collections import defaultdict

import reward as tr

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
MAX_STEPS  = 4e6
CLIP_GRAD = 0.5

<h2>Use CUDA</h2>

In [4]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [5]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()

In [6]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

<h1>Soft Actor-Critic: Off-Policy Maximum Entropy Deep Reinforcement Learning with a Stochastic Actor</h1>
<h2><a href="https://arxiv.org/abs/1801.01290">Arxiv</a></h2>

In [7]:
class ValueNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim, init_w=3e-3):
        super(ValueNetwork, self).__init__()
        
        self.linear1 = nn.Linear(state_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, 1)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
        
        
class SoftQNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(SoftQNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, 1)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
        
        
class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(PolicyNetwork, self).__init__()
        
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        
        self.mean_linear = nn.Linear(hidden_size, num_actions)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size, num_actions)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        
        mean    = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        return mean, log_std
    
    def evaluate(self, state, epsilon=1e-6):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        normal = Normal(mean, std)
        z = normal.sample()
        action = torch.tanh(z)
        
        log_prob = normal.log_prob(z) - torch.log(1 - action.pow(2) + epsilon)
        log_prob = log_prob.sum(-1, keepdim=True)
        
        return action, log_prob, z, mean, log_std
        
    
    def get_action(self, state, step):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        normal = Normal(mean, std)
        z      = normal.sample()
        action = torch.tanh(z)
        
        # TODO
        action  = action.detach().cpu().numpy()
        return action

In [8]:
def soft_q_update(batch, 
           gamma=0.99,
           mean_lambda=1e-3,
           std_lambda=1e-3,
           z_lambda=0.0,
           soft_tau=1e-2,
           frame_idx=0,
           logger=None,            
          ):
    state, action, reward, next_state, done = batch.state_t, batch.action, batch.reward, batch.state_tp1, batch.done

    expected_q_value = soft_q_net(state, action)
    expected_value   = value_net(state)
    new_action, log_prob, z, mean, log_std = policy_net.evaluate(state)


    target_value = target_value_net(next_state)
    next_q_value = reward + (1 - done) * gamma * target_value
    q_value_loss = soft_q_criterion(expected_q_value, next_q_value.detach())

    expected_new_q_value = soft_q_net(state, new_action)
    next_value = expected_new_q_value - log_prob
    value_loss = value_criterion(expected_value, next_value.detach())

    log_prob_target = expected_new_q_value - expected_value
    policy_loss = (log_prob * (log_prob - log_prob_target).detach()).mean()
    

    mean_loss = mean_lambda * mean.pow(2).mean()
    std_loss  = std_lambda  * log_std.pow(2).mean()
    z_loss    = z_lambda    * z.pow(2).sum(1).mean()

    policy_loss += mean_loss + std_loss + z_loss

    soft_q_optimizer.zero_grad()
    q_value_loss.backward()
    q_grad = torch.nn.utils.clip_grad_norm_(soft_q_net.parameters(), CLIP_GRAD)
    soft_q_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward()
    v_grad = torch.nn.utils.clip_grad_norm_(value_net.parameters(), CLIP_GRAD)
    value_optimizer.step()

    policy_optimizer.zero_grad()
    policy_loss.backward()
    p_grad = torch.nn.utils.clip_grad_norm_(policy_net.parameters(), CLIP_GRAD)
    policy_optimizer.step()
    
    
    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - soft_tau) + param.data * soft_tau
        )

    if frame_idx % 1000 == 0:
        # Store information
        logger.add_log('policy/loss', policy_loss)
        logger.add_log('v/loss', value_loss)
        logger.add_log('q/loss', q_value_loss)
        
        logger.add_log('policy/grad', p_grad)
        logger.add_log('v/grad', v_grad)
        logger.add_log('q/grad', q_grad)
        
        logger.add_histogram('policy/log_prob', log_prob)
        logger.add_histogram('V', expected_value)
        logger.add_histogram('Q', expected_q_value)

In [9]:
env = tr.envs.GymEnv("HalfCheetah-v2")
env = tr.envs.wrappers.ActionBound(env)
runner = tr.runners.SingleRunner(env)
batcher = tr.batchers.ReplayBatcher(
    runner=runner,
    batch_size=128,
    replay_buffer_maxlen=1e6,
    learning_freq=1,
    grad_steps_per_batch=1,
    init_replays=500 / 1e6,
)

action_dim = env.action_info.shape[0]
state_dim = env.state_info.shape[0]

Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']


100%|██████████| 500/500 [00:00<00:00, 4266.41it/s]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Populating Replay Buffer...


In [10]:
hidden_dim = 256

value_net        = ValueNetwork(state_dim, hidden_dim).to(device)
target_value_net = ValueNetwork(state_dim, hidden_dim).to(device)

soft_q_net = SoftQNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)
    

value_criterion  = nn.MSELoss()
soft_q_criterion = nn.MSELoss()

value_lr  = 3e-4
soft_q_lr = 3e-4
policy_lr = 3e-4

value_optimizer  = optim.Adam(value_net.parameters(), lr=value_lr)
soft_q_optimizer = optim.Adam(soft_q_net.parameters(), lr=soft_q_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)

In [11]:
logger = U.Logger('logs/halfcheetah/batcher-grad0.5-v0-1')

Writing logs to: logs/halfcheetah/batcher-grad0.5-v0-1


In [ ]:
for batch in batcher.get_batches(MAX_STEPS, select_action_fn=policy_net.get_action):
    # Manipulate batch, calculate adv, etc
    batch = batch.to_tensor().concat_batch()
    # Optimization step
    soft_q_update(batch, soft_tau=0.005, logger=logger)
    
    # Write logs
    if (batcher.num_steps) % 1000 == 0:
        batcher.write_logs(logger)
        logger.log(step=batcher.num_steps)

  0%|          | 996/4.00M [00:09<21:07:00, 52.6it/s]/home/quazar/anaconda3/envs/reward_up/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/quazar/anaconda3/envs/reward_up/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 1.01k/4.00M [00:10<24:42:24, 45.0it/s]


--------------------------------------------------------------
policy/loss                                            |  1.47
v/loss                                                 |  1.40
q/loss                                                 |  0.45
policy/grad                                            |  3.04
v/grad                                                 | 13.58
q/grad                                                 |  3.43
Env/Reward/Episode (New)                               |   nan
Env/Length/Episode (New)                               |   nan
Env/Reward/Episode (Last 50)                           |   nan
Env/Length/Episode (Last 50)                           |   nan
--------------------------------------------------------------


  0%|          | 2.01k/4.00M [00:29<22:35:55, 49.1it/s]


--------------------------------------------------------------
policy/loss                                          |    0.47
v/loss                                               |    1.18
q/loss                                               |    1.19
policy/grad                                          |    2.62
v/grad                                               |   32.13
q/grad                                               |   18.48
Env/Reward/Episode (New)                             | -357.24
Env/Length/Episode (New)                             | 1500.00
Env/Reward/Episode (Last 50)                         | -357.24
Env/Length/Episode (Last 50)                         | 1500.00
--------------------------------------------------------------


  0%|          | 3.01k/4.00M [00:48<21:50:23, 50.8it/s]

...

  3%|▎         | 129k/4.00M [42:19<24:00:47, 44.8it/s]


--------------------------------------------------------------
policy/loss                                          |    0.11
v/loss                                               |    4.71
q/loss                                               |  136.80
policy/grad                                          |    3.16
v/grad                                               | 1025.14
q/grad                                               | 1029.48
Env/Reward/Episode (New)                             | -197.11
Env/Length/Episode (New)                             | 1000.00
Env/Reward/Episode (Last 50)                         | -175.97
Env/Length/Episode (Last 50)                         | 1000.00
--------------------------------------------------------------


  3%|▎         | 130k/4.00M [42:40<22:00:50, 48.8it/s]


--------------------------------------------------------------
policy/loss                                          |    0.21
v/loss                                               |    4.13
q/loss                                               |  139.09
policy/grad                                          |    2.93
v/grad                                               |  990.05
q/grad                                               | 1007.75
Env/Reward/Episode (New)                             | -159.45
Env/Length/Episode (New)                             | 1000.00
Env/Reward/Episode (Last 50)                         | -174.85
Env/Length/Episode (Last 50)                         | 1000.00
--------------------------------------------------------------


  3%|▎         | 131k/4.00M [42:51<25:20:46, 42.4it/s]